In [1]:
import os
import openai
import json
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import time


# Connect to ChatGpt Api

In [2]:
openai.organization = os.getenv("CHATGPTORG")
openai.api_key = os.getenv("CHATGPTKEY")
openai.Model.list()

<OpenAIObject list at 0x1374dbea0> JSON: {
  "data": [
    {
      "created": 1677532384,
      "id": "whisper-1",
      "object": "model",
      "owned_by": "openai-internal",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1683912666,
          "group": null,
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "whisper-1"
    },
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
         

## Assign Categories

In [ ]:
# open our test dataframe from the csv
df = pd.read_csv('data/test_datasets/random/random_sample_100_v2.csv')
df.head()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,priceCurrency_right,specTableContent_right,cluster_id_right,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction
0,70108616,Maxxis,Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead,The new incarnation of the Minion DHR. Ready to shred any line.,110.00,CAD,NaN,1084991,45040021,Maxxis,...,USD,NaN,1084991,70108616#45040021,1,False,"[-4.3633804321, 4.6556377411]",1,[1.0],1
1,67977820,NaN,Canon CLI251XL High Yield Black Inkjet Cartridge Remanufactured,"Crazy Inkjets is a leading supplier of high quality printing supplies for your Canon CLI251XL printer cartridge. CrazyInkjets products are guaranteed to meet or exceed the quality, reliability and yield standards of the original equipment remanufacturer. The units are tested to ensure compliance...",6.95,USD,NaN,767463,69787972,Samsung,...,AUD,NaN,1467531,67977820#69787972,0,True,"[3.8366084099, -4.2942962646]",0,[0.0],0
2,79859336,Crucial,Crucial Crucial Memory 4GB DDR4 2666 Unbuffered CT4G4SFS8266,NaN,34.99,CAD,NaN,1892167,65646040,Crucial,...,TRY,NaN,1892167,79859336#65646040,1,False,"[-4.363576889, 4.6508393288]",1,[1.0],1
3,30368884,NaN,"Crucial MX500 250GB 2.5"" SATA III","MX500 250GB SATA 2.5-inch, SATA 6.0Gb/s, 560 MB/s Read, 510 MB/s Write",43.59,EUR,NaN,672125,86893846,NaN,...,ZAR,NaN,1409979,30368884#86893846,0,False,"[3.8347194195, -4.2961273193]",0,[0.0],0
4,82078171,NaN,Intel Core I3 7th Gen 7100 3.90 Ghz; 2 Core 4 Thread; 3 Mb Smartcache; 51 W Tdp; Lga 1151 S R35 C,"ntel i3-7100, Core. Processor family: 7th gen Intel® Core™ i3, Processor frequency: 3.9 GHz, Processor socket: LGA 1151 (Socket H4). Memory channels: Dual, Maximum internal memory supported by processor: 64 GB, Memory types supported by processor: DDR3L-SDRAM,DDR4-SDRAM. On-board graphics adapte...",2.899E3,ZAR,NaN,443612,6914049,WESTERN DIGITAL,...,EUR,NaN,549556,82078171#6914049,0,False,"[3.8386721611, -4.3010449409]",0,[0.0],0


In [ ]:
df.shape

(100, 23)

In [ ]:
# create a new column for the chatbot response
df['chatbot_response'] = ''
df['chatbot_response_raw'] = ''
df['chatbot_question'] = ''

In [ ]:
# loop through the dataframe and send the messages to the chatbot
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    message = [
        {"role": "system", "content": "You are a helpful assistant that assigns products to product categories. Please provide them in this format: product 1: category product 2: category They should be at a high level so things like electronics, outdoors etc."},
        {"role": "user",
            "content": f" product 1: {row['title_left']}, product 2: {row['title_right']}"},
    ]
    error = ""
    for i in range(3):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4-0613",
                messages=message,
                max_tokens=40,
                temperature=0.0,
            )
            # if the above call succeeds, break out of the loop and continue
            break
        except Exception as e:
            error = e
            print(f"Error: {e}")
            # if the call fails, wait for one minute before retrying
            time.sleep(60)

    # check if the response was set successfully
    if response:
        # your code here
        df.at[index, 'chatbot_response_raw'] = response
        df.at[index, 'chatbot_response'] = response['choices'][0]['message']['content']
        df.at[index, 'chatbot_question'] = message
        print(index, response['choices'][0]['message']['content'])
    else:
        df.at[index, 'chatbot_response_raw'] = error
        df.at[index, 'chatbot_response'] = -1
        df.at[index, 'chatbot_question'] = message
        print("Error: response was not set successfully")

  0%|          | 0/100 [00:00<?, ?it/s]

0 product 1: Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead: Cycling Equipment, product 2: Maxxis Minion DHR II 3
1 product 1: Office Supplies, product 2: Office Supplies
2 product 1: Crucial Crucial Memory 4GB DDR4 2666 Unbuffered CT4G4SFS8266: Electronics
product 2: CRUCIAL CT4
3 product 1: Crucial MX500 250GB 2.5" SATA III: Electronics
product 2: JABARA Evolve 65 with Link 370 USB - Mono:
4 product 1: Intel Core I3 7th Gen 7100 3.90 Ghz; 2 Core 4 Thread; 3 Mb Smartcache; 51 W Tdp
5 Product 1: Tissot Seastar 1000 T120.417.11.051.00: Watches
Product 2: Samsung Portable SSD T7 Touch MU-PC1
6 product 1: Samsung 970 EVO SSD M.2 2280 - 1TB SSD - 1 TB - M.2 2280 (80mm) - PCI Express 
7 product 1: Ilford Ilford FP4+ 125 120 B&W Film - Single Roll: Photography
product 2: Canon EF-S 18-200mm f3.5
8 product 1: iiyama 27' ETE, ULTRA SLIM LINE, 2560x1440 WQHD, IPS, 5ms, FreeSync, 13
9 product 1: Cycling Equipment, product 2: Cycling Equipment
10 product 1: Kingston Technology DataTraveler SE9 G

In [ ]:
df.head()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question
0,70108616,Maxxis,Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead,The new incarnation of the Minion DHR. Ready to shred any line.,110.00,CAD,NaN,1084991,45040021,Maxxis,...,70108616#45040021,1,False,"[-4.3633804321, 4.6556377411]",1,[1.0],1,"product 1: Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead: Cycling Equipment, product 2: Maxxis Minion DHR II 3","{'id': 'chatcmpl-7a2PJXD30RMVmVvOJrSzcqSRJ5ouz', 'object': 'chat.completion', 'created': 1688823417, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': { ""content"": ""product 1: Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead: Cycling Equipment, product 2: Maxxis Minion DHR II 3"", ""ro...","[{'role': 'system', 'content': 'You are a helpful assistant that assigns products to product categories. Please provide them in this format: product 1: category product 2: category They should be at a high level so things like electronics, outdoors etc.'}, {'role': 'user', 'content': ' product 1..."
1,67977820,NaN,Canon CLI251XL High Yield Black Inkjet Cartridge Remanufactured,"Crazy Inkjets is a leading supplier of high quality printing supplies for your Canon CLI251XL printer cartridge. CrazyInkjets products are guaranteed to meet or exceed the quality, reliability and yield standards of the original equipment remanufacturer. The units are tested to ensure compliance...",6.95,USD,NaN,767463,69787972,Samsung,...,67977820#69787972,0,True,"[3.8366084099, -4.2942962646]",0,[0.0],0,"product 1: Office Supplies, product 2: Office Supplies","{'id': 'chatcmpl-7a2PNNQOnhJRFKKMVzuqggncuJGVG', 'object': 'chat.completion', 'created': 1688823421, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': { ""content"": ""product 1: Office Supplies, product 2: Office Supplies"", ""role"": ""assistant"" }, 'finish_reason': 'stop'}], 'usage': {'...","[{'role': 'system', 'content': 'You are a helpful assistant that assigns products to product categories. Please provide them in this format: product 1: category product 2: category They should be at a high level so things like electronics, outdoors etc.'}, {'role': 'user', 'content': ' product 1..."
2,79859336,Crucial,Crucial Crucial Memory 4GB DDR4 2666 Unbuffered CT4G4SFS8266,NaN,34.99,CAD,NaN,1892167,65646040,Crucial,...,79859336#65646040,1,False,"[-4.363576889, 4.6508393288]",1,[1.0],1,product 1: Crucial Crucial Memory 4GB DDR4 2666 Unbuffered CT4G4SFS8266: Electronics\nproduct 2: CRUCIAL CT4,"{'id': 'chatcmpl-7a2POoegCmNDoLsZag4KOIztZj4mg', 'object': 'chat.completion', 'created': 1688823422, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': { ""content"": ""product 1: Crucial Crucial Memory 4GB DDR4 2666 Unbuffered CT4G4SFS8266: Electronics\nproduct 2: CRUCIAL CT4"", ""role"":...","[{'role': 'system', 'content': 'You are a helpful assistant that assigns products to product categories. Please provide them in this format: product 1: category product 2: category They should be at a high level so things like electronics, outdoors etc.'}, {'role': 'user', 'content': ' product 1..."
3,30368884,NaN,"Crucial MX500 250GB 2.5"" SATA III","MX500 250GB SATA 2.5-inch, SATA 6.0Gb/s, 560 MB/s Read, 510 MB/s Write",43.59,EUR,NaN,672125,86893846,NaN,...,30368884#86893846,0,False,"[3.8347194195, -4.2961273193]",0,[0.0],0,"product 1: Crucial MX500 250GB 2.5"" SATA III: Electronics\nproduct 2: JABARA Evolve 65 with Link 370 USB - Mono:","{'id': 'chatcmpl-7a2PSC2hJ9ojRg1tvYYBagWh0Fcuh', 'object': 'chat.completion', 'created': 1688823426, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': { ""content"": ""product 1: Crucial MX500 250GB 2.5\"" SATA III: Electronics\nproduct 2: JABARA Evolve 65 with Link 370 USB - Mono:"", ""r...","[{'role':

In [ ]:
# get the number of rows were the chatbot respionse is not empty
df[df['chatbot_response'] != ''].shape

(100, 26)

In [ ]:
df.tail()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question
95,53525303,Ryze,Ryze Tello Battery Charging Hub G1CH,NaN,129,DKK,NaN,1555405,79449686,NaN,...,53525303#79449686,0,True,"[3.6632392406000003, -4.1180138588]",0,[0.0],0,"product 1: Ryze Tello Battery Charging Hub G1CH: Electronics, product 2: AirPods with Wireless Charging Case: Electronics","{'id': 'chatcmpl-7a2U4IyKM563DbZAJkfvcy2LMdScH', 'object': 'chat.completion', 'created': 1688823712, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': { ""content"": ""product 1: Ryze Tello Battery Charging Hub G1CH: Electronics, product 2: AirPods with Wireless Charging Case: Electronic...","[{'role': 'system', 'content': 'You are a helpful assistant that assigns products to product categories. Please provide them in this format: product 1: category product 2: category They should be at a high level so things like electronics, outdoors etc.'}, {'role': 'user', 'content': ' product 1..."
96,81415527,NaN,2MP 25X Network IR PTZ Camera,"1/2.8\"" progressive scan CMOSUp to 1920 × 1080@30fps resolutionMin. illumination: Color: 0.005 Lux @(F1.6, AGC ON) B/W: 0.001 Lux @(F1.6, AGC ON) 0 Lux with IR25× optical zoom, 16× digital zoomWDR, HLC, BLC, 3D DNR, Defog, EIS, Regional Exposure, Regional FocusUp to 150 m IR distance24 VAC & Hi-...","Incl. BTW€1.035,76",EUR,NaN,2694265,46810114,NaN,...,81415527#46810114,1,False,"[-4.1793274879, 4.4736194611]",1,[1.0],1,product 1: 2MP 25X Network IR PTZ Camera: Electronics\nproduct 2: HIKVISION 2MP 25X NETWORK IR SPEED DOME CAMERA DS-2,"{'id': 'chatcmpl-7a2U7eSkdNrFqE25zBYz9zzBAxjJu', 'object': 'chat.completion', 'created': 1688823715, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': { ""content"": ""product 1: 2MP 25X Network IR PTZ Camera: Electronics\nproduct 2: HIKVISION 2MP 25X NETWORK IR SPEED DOME CAMERA DS-2"", ...","[{'role': 'system', 'content': 'You are a helpful assistant that assigns products to product categories. Please provide them in this format: product 1: category product 2: category They should be at a high level so things like electronics, outdoors etc.'}, {'role': 'user', 'content': ' product 1..."
97,53053823,NaN,Canon EF-S 18-135mm f/3.5-5.6 IS USM Zoom Lens,"The Canon EF-S 18-135mm f/3.5-5.6 IS USM Zoom Lens is great for photos and movies. With improved Image Stabilisation, it allows hand held shooting even in lower light conditions.Features 16 elements from 12 groups18-135mm Focal LengthBuilt-in Image StabiliserNano USM technologyMaximum magnificat...",799,AUD,NaN,390091,45511817,NaN,...,53053823#45511817,0,True,"[3.7480974197, -4.2194314003]",0,[0.0],0,product 1: Canon EF-S 18-135mm f/3.5-5.6 IS USM Zoom Lens: Electronics\nproduct 2: Nikon Nikkor AF-S 24,"{'id': 'chatcmpl-7a2UAdB9lNQJjjIT15dxMykW1DHn6', 'object': 'chat.completion', 'created': 1688823718, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': { ""content"": ""product 1: Canon EF-S 18-135mm f/3.5-5.6 IS USM Zoom Lens: Electronics\nproduct 2: Nikon Nikkor AF-S 24"", ""role"": ""ass...","[{'role': 'system', 'content': 'You are a helpful assistant that assigns products to product categories. Please provide them in this format: product 1: category product 2: category They should be at a high level so things like electronics, outdoors etc.'}, {'role': 'user', 'content': ' product 1..."
98,72039756,NaN,"Cooler Master Chassis, MasterBox Lite 5 Tower",NaN,64.52,USD,NaN,1006147,43470746,NaN,...,72039756#43470746,1,False,"[-4.3535614014, 4.6426811218]",1,[1.0],1,"product 1: Cooler Master Chassis, MasterBox Lite 5 Tower: Computer Hardware\nproduct 2: CASE MID-TOWER NO PSU MASTERBOX LITE 5 3USB3 BLACK","{'id': 'chatcmpl-7a2UEQFZQ5peIqDXOWceYq1w3g06j', 'object': 'chat.completion', 'created': 16888237

In [ ]:
product_categories = [
"Cycling Equipment",
"Office Supplies",
"Electronics",
"Watches",
"Photography",
"Computer Hardware",
"Printer Accessories",
"Computer Accessories",
"Computer Storage",
"Trading Card Games",
"Outdoors",
"Musical Instruments",
"Outdoor Cooking Equipment",
"Camera Lenses",
"Photography Accessories",
"Stationery",
"Computer Components"]



# remove duplicates from product categories 
_product_categories = []
for product in product_categories:
    if product in _product_categories:
        continue
    _product_categories.append(product)

counts = {}
_product_categories

['Cycling Equipment',
 'Office Supplies',
 'Electronics',
 'Watches',
 'Photography',
 'Computer Hardware',
 'Printer Accessories',
 'Computer Accessories',
 'Computer Storage',
 'Trading Card Games',
 'Outdoors',
 'Musical Instruments',
 'Outdoor Cooking Equipment',
 'Camera Lenses',
 'Photography Accessories',
 'Stationery',
 'Computer Components']

In [ ]:
def clean_response(response):
    categories = []
    for product in _product_categories:
        if product.lower() in response.lower():
            # count how often it is in response
            counts[product] = counts.get(product, 0) + response.lower().count(product.lower())

In [ ]:
# add a coloumn with a clean version of the chatbot response if the chatresponse contains Yes/yes it will be 1 else if it contains No/no it will be 0 if it is 0 or 1 
df['chatbot_response_clean'] = df['chatbot_response'].apply(lambda x: clean_response(x))

In [ ]:
counts

{'Cycling Equipment': 6,
 'Office Supplies': 10,
 'Electronics': 59,
 'Watches': 11,
 'Photography': 4,
 'Printer Accessories': 8,
 'Computer Hardware': 7,
 'Computer Components': 5,
 'Outdoors': 2,
 'Computer Accessories': 3,
 'Computer Storage': 2,
 'Trading Card Games': 2,
 'Stationery': 2,
 'Musical Instruments': 3,
 'Outdoor Cooking Equipment': 1,
 'Camera Lenses': 1,
 'Photography Accessories': 1}

easy:
{'Cycling Equipment': 6,
 'Office Supplies': 18,
 'Electronics': 59,
 'Watches': 11,
 'Computer Hardware': 16,
 'Outdoors': 4,
 'Trading Card Games': 2,
 'Musical Instruments': 4,
 'Photography Equipment': 6}

middle: 
{'Electronics': 71,
 'Watches': 24,
 'Musical Instruments': 4,
 'Cycling Equipment': 10,
 'Office Supplies': 15,
 'Computer Hardware': 14,
 'Photography Equipment': 7}

hard:
{'Watches': 26,
 'Electronics': 56,
 'Musical Instruments': 10,
 'Computer Hardware': 4,
 'Photography Equipment': 16,
 'Cycling Equipment': 16,
 'Office Supplies': 20,
 'Outdoors': 10,}

In [ ]:
# get the current date and time
now = datetime.now()

# save the dataframe as a json file
df.to_json(f'data/results/chat_gpt/medium/{now}_simple_promt_all.json')

In [110]:
categories_df = pd.read_json("paper_data/product_categories/hard_categories.json")
categories_df

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question,chatbot_response_clean
0,1840,98621092,None,TAG Heuer Monaco Chronograph Calibre 11 Automatic,Design inspired by the watch worn by Steve McQ...,"£4,195.00",None,NaN,556904,2004031,...,1,False,"[3.7184028625, -4.1772656441]",0,[1.0],1,product 1: TAG Heuer Monaco Chronograph Calibr...,{'id': 'chatcmpl-7a2D48TBSy9MySB9BhZzUxMeWKdax...,"[{'role': 'system', 'content': 'You are a help...",NaN
1,710,47887901,None,Apple AirPods (2nd Generation) Bluetooth Earbu...,Staples.com: Apple AirPods MRXJ2AM/A In the Ea...,199,USD,NaN,1994510,49097388,...,0,True,"[-1.7439045906000001, 1.717010498]",1,[0.0],0,product 1: Apple AirPods (2nd Generation) Blue...,{'id': 'chatcmpl-7a2D7iGz7uZSjQ3SCnyYbCeIFykeY...,"[{'role': 'system', 'content': 'You are a help...",NaN
2,2065,60950283,None,Fujifilm Instax Square Instant Film - 10 Photos,The Fujifilm Instax Square Instant Film is for...,11.99,GBP,NaN,1457721,25935656,...,0,True,"[-2.1758391857, 2.2021055222]",1,[0.0],0,product 1: Fujifilm Instax Square Instant Film...,{'id': 'chatcmpl-7a2DBt3ZspFNrgylhL4vAEETJHHRh...,"[{'role': 'system', 'content': 'You are a help...",NaN
3,3869,435472,None,Evans EMAD Clear Bass Drum Head 20 Inch,"20"" drum head made using a single ply of 10mil...",46.06,USD,NaN,800940,55131100,...,1,False,"[3.2268047333, -3.6520571709]",0,[1.0],1,product 1: Evans EMAD Clear Bass Drum Head 20 ...,{'id': 'chatcmpl-7a2DFLNDxFjOxbmYZ29IByW3ZBmfL...,"[{'role': 'system', 'content': 'You are a help...",NaN
4,1787,88641225,None,"8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...","8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...",74.99,CAD,NaN,288081,86250508,...,0,True,"[-2.2421159744, 2.2641017437]",1,[0.0],0,product 1: Electronics\nproduct 2: Electronics,{'id': 'chatcmpl-7a2DHkjQSyCCVTYHdhQxfbw1zjVkC...,"[{'role': 'system', 'content': 'You are a help...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2820,31581824,SHIMANO,SPD-SL SH 11 - tacchette,The Shimano SM-SH11 cleats serve as a connecti...,15.33,EUR,NaN,532781,75286119,...,0,True,"[-4.3649616241, 4.6570587158]",1,[0.0],0,product 1: SPD-SL SH 11 - tacchette: Cycling A...,{'id': 'chatcmpl-7a2Hgl5WRIRCrLiCBmFmfBoyACEHx...,"[{'role': 'system', 'content': 'You are a help...",NaN
96,4223,6002122,None,LOGITECH - STEREO HEADSET H151 ANALOG - EMEAIN...,LOGITECH STEREO HEADSET H151 ANALOG - EMEAIN A...,None,None,NaN,122031,31273059,...,0,True,"[-4.3757534027, 4.6587324142]",1,[0.0],0,product 1: LOGITECH - STEREO HEADSET H151 ANAL...,{'id': 'chatcmpl-7a2Hj1kCx5cz6UXgLoEcrhdYwtAyy...,"[{'role': 'system', 'content': 'You are a help...",NaN
97,2073,393407,Audemars Piguet,Audemars Piguet Royal Oak Chronograph,Audemars Piguet Royal Oak,36560,USD,NaN,607611,98604797,...,0,True,"[-4.1834125519, 4.471534729]",1,[0.0],0,product 1: Audemars Piguet Royal Oak Chronogra...,{'id': 'chatcmpl-7a2HofHRE51pBqgLPCuAgXCTyDcXD...,"[{'role': 'system', 'content': 'You are a help...",NaN
98,3917,89292552,None,Shimano SPD SL Cleat Set,Shimano cleats come in 3 different colours the...,39.95,AUD,NaN,431550,75045785,...,0,True,"[-3.1417820454, 3.3016252518]",1,[0.0],0,product 1: Shimano SPD SL Cleat Set: Sports & ...,{'id': 'chatcmpl-7a2HrlB74yLvpjrOOV23dxOjPdd0n...,"[{'role': 'system', 'content': 'You are a help...",NaN


In [125]:
gpt_answers = pd.read_json("data/results/aleph_alpha/2023-05-09 16:09:07.988052_simple_promt_hard.json")

In [126]:
# Add the chatbot_response couloum of the categories_df to the gpt_answers as a categorie couloum
gpt_answers['category'] = categories_df['chatbot_response']
gpt_answers.head()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question,chatbot_response_clean,category
0,1840,98621092,None,TAG Heuer Monaco Chronograph Calibre 11 Automatic,Design inspired by the watch worn by Steve McQ...,"£4,195.00",None,NaN,556904,2004031,...,False,"[3.7184028625, -4.1772656441]",0,[1.0],1,", Calibre 11 Automatic.","{'completions': [{'completion': ', Calibre 11 ...",Do the following two product decriptions match...,-1,product 1: TAG Heuer Monaco Chronograph Calibr...
1,710,47887901,None,Apple AirPods (2nd Generation) Bluetooth Earbu...,Staples.com: Apple AirPods MRXJ2AM/A In the Ea...,199,USD,NaN,1994510,49097388,...,True,"[-1.7439045906000001, 1.717010498]",1,[0.0],0,\n\nA:\n\n,"{'completions': [{'completion': ' A: ', 'fin...",Do the following two product decriptions match...,-1,product 1: Apple AirPods (2nd Generation) Blue...
2,2065,60950283,None,Fujifilm Instax Square Instant Film - 10 Photos,The Fujifilm Instax Square Instant Film is for...,11.99,GBP,NaN,1457721,25935656,...,True,"[-2.1758391857, 2.2021055222]",1,[0.0],0,",\n\nA: Yes","{'completions': [{'completion': ', A: Yes', '...",Do the following two product decriptions match...,1,product 1: Fujifilm Instax Square Instant Film...
3,3869,435472,None,Evans EMAD Clear Bass Drum Head 20 Inch,"20"" drum head made using a single ply of 10mil...",46.06,USD,NaN,800940,55131100,...,False,"[3.2268047333, -3.6520571709]",0,[1.0],1,Drum Head 20 Inch.\n,{'completions': [{'completion': ' Drum Head 20...,Do the following two product decriptions match...,-1,product 1: Evans EMAD Clear Bass Drum Head 20 ...
4,1787,88641225,None,"8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...","8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...",74.99,CAD,NaN,288081,86250508,...,True,"[-2.2421159744, 2.2641017437]",1,[0.0],0,- 1.5V -,"{'completions': [{'completion': ' - 1.5V -', '...",Do the following two product decriptions match...,-1,product 1: Electronics\nproduct 2: Electronics


In [45]:
categories = ['Cycling Equipment',
 'Office Supplies',
 'Electronics',
 'Watches',
 'Photography',
 'Computer Hardware',
 'Printer Accessories',
 'Computer Accessories',
 'Computer Storage',
 'Trading Card Games',
 'Outdoors',
 'Musical Instruments',
 'Outdoor Cooking Equipment',
 'Camera Lenses',
 'Photography Accessories',
 'Stationery',
 'Computer Components']

def clean_category(title_left, title_right, category):
    category = category.replace(title_left, "")
    category = category.replace(title_right, "")
    category = category.replace("product 1:", "")
    category = category.replace("product 2", "")
    for cat in categories:
        if cat in category:
            return cat
    return "No category"

In [127]:
gpt_answers['clean_category'] = gpt_answers.apply(lambda row: clean_category(row["title_left"], row["title_right"], row["category"]), axis=1)

In [128]:
# Group by category and calculate total count
category_counts = gpt_answers.groupby('clean_category').size().reset_index(name='category_count')

# Calculate count of invalid responses per category
invalid_counts = gpt_answers[gpt_answers['chatbot_response_clean'] == -1].groupby('clean_category').size().reset_index(name='invalid_counts')

# Calculate count of wrong answers per category
wrong_counts = gpt_answers[gpt_answers['chatbot_response_clean'] != gpt_answers['label'] & gpt_answers['chatbot_response_clean'] != -1].groupby('clean_category').size().reset_index(name='wrong_counts')

# Merge the counts and category counts DataFrames
merged_df = category_counts.merge(invalid_counts, on='clean_category', how='left').merge(wrong_counts, on='clean_category', how='left')

# Calculate error rates
merged_df['invalid_rate'] = merged_df['invalid_counts'] / merged_df['category_count']
merged_df['wrong_rate'] = merged_df['wrong_counts'] / merged_df['category_count']

# Fill NaN values with 0
merged_df.fillna(0, inplace=True)

# Print the resulting DataFrame
merged_df["model"] = "Aleph Alpha"
merged_df["benchmark"] = "challenging"
merged_df


,clean_category,category_count,invalid_counts,wrong_counts,invalid_rate,wrong_rate,model,benchmark
0,Camera Lenses,1,1.0,1,1.000000,1.000000,Aleph Alpha,challenging
1,Computer Accessories,1,1.0,1,1.000000,1.000000,Aleph Alpha,challenging
2,Computer Hardware,2,2.0,2,1.000000,1.000000,Aleph Alpha,challenging
3,Cycling Equipment,6,6.0,6,1.000000,1.000000,Aleph Alpha,challenging
4,Electronics,39,38.0,38,0.974359,0.974359,Aleph Alpha,challenging
5,Musical Instruments,6,6.0,6,1.000000,1.000000,Aleph Alpha,challenging
6,No category,7,7.0,7,1.000000,1.000000,Aleph Alpha,challenging
7,Office Supplies,4,4.0,4,1.000000,1.000000,Aleph Alpha,challenging
8,Outdoors,6,4.0,4,0.666667,0.666667,Aleph Alpha,challenging
9,Photography,8,5.0,8,0.625000,1.000000,Aleph Alpha,challenging


In [130]:
scores_df = pd.concat([scores_df, merged_df], ignore_index=True)

In [131]:
scores_df

,clean_category,category_count,invalid_counts,wrong_counts,invalid_rate,wrong_rate,model,benchmark
0,Computer Accessories,1,0.0,0.0,0.000000,0.000000,ChatGPT,moderate
1,Computer Components,2,0.0,0.0,0.000000,0.000000,ChatGPT,moderate
2,Computer Hardware,7,0.0,0.0,0.000000,0.000000,ChatGPT,moderate
3,Cycling Equipment,7,0.0,1.0,0.000000,0.142857,ChatGPT,moderate
4,Electronics,49,0.0,16.0,0.000000,0.326531,ChatGPT,moderate
...,...,...,...,...,...,...,...,...
121,Outdoors,6,4.0,4.0,0.666667,0.666667,Aleph Alpha,challenging
122,Photography,8,5.0,8.0,0.625000,1.000000,Aleph Alpha,challenging
123,Printer Accessories,5,4.0,4.0,0.800000,0.800000,Aleph Alpha,challenging
124,Stationery,1,0.0,1.0,0.000000,1.000000,Aleph Alpha,challenging


In [69]:
scores_df = merged_df
scores_df

,clean_category,category_count,invalid_counts,wrong_counts,invalid_rate,wrong_rate,model,benchmark
0,Computer Accessories,1,0.0,0.0,0.0,0.000000,ChatGPT,moderate
1,Computer Components,2,0.0,0.0,0.0,0.000000,ChatGPT,moderate
2,Computer Hardware,7,0.0,0.0,0.0,0.000000,ChatGPT,moderate
3,Cycling Equipment,7,0.0,1.0,0.0,0.142857,ChatGPT,moderate
4,Electronics,49,0.0,16.0,0.0,0.326531,ChatGPT,moderate
5,Musical Instruments,2,0.0,1.0,0.0,0.500000,ChatGPT,moderate
6,No category,3,0.0,0.0,0.0,0.000000,ChatGPT,moderate
7,Office Supplies,7,0.0,1.0,0.0,0.142857,ChatGPT,moderate
8,Photography,5,0.0,2.0,0.0,0.400000,ChatGPT,moderate
9,Printer Accessories,4,0.0,1.0,0.0,0.250000,ChatGPT,moderate


In [136]:
scores_df.to_json("paper_data/product_categories/category_overview_v2.json")

In [135]:
scores_df["wrong_rate"] = scores_df["wrong_rate"] / scores_df["category_count"]
scores_df

,clean_category,category_count,invalid_counts,wrong_counts,invalid_rate,wrong_rate,model,benchmark
0,Computer Accessories,1,0.0,0.0,0.000000,0.000000,ChatGPT,moderate
1,Computer Components,2,0.0,0.0,0.000000,0.000000,ChatGPT,moderate
2,Computer Hardware,7,0.0,0.0,0.000000,0.000000,ChatGPT,moderate
3,Cycling Equipment,7,0.0,1.0,0.000000,0.020408,ChatGPT,moderate
4,Electronics,49,0.0,16.0,0.000000,0.006664,ChatGPT,moderate
...,...,...,...,...,...,...,...,...
121,Outdoors,6,4.0,0.0,0.111111,0.111111,Aleph Alpha,challenging
122,Photography,8,5.0,3.0,0.078125,0.125000,Aleph Alpha,challenging
123,Printer Accessories,5,4.0,0.0,0.160000,0.160000,Aleph Alpha,challenging
124,Stationery,1,0.0,1.0,0.000000,1.000000,Aleph Alpha,challenging


In [149]:
# Create a new DataFrame with the desired columns
table_df = pd.DataFrame(columns=['clean_category', 'category_count', 'invalid_counts', 'wrong_counts', 'invalid_rate', 'wrong_rate', 'model', 'benchmark'])

# Get unique categories and models
unique_categories = scores_df['clean_category'].unique()
unique_models = scores_df["model"].unique()

# Iterate over categories and models to populate the table
for category in unique_categories:
    for model in unique_models:
        # Get the specific row for the category and model
        category_row = scores_df[scores_df['clean_category'] == category]
        model_row = category_row[category_row['model'] == model]
        
        # Get the benchmark values (easy, moderate, challenging)
        easy_values = [model_row['invalid_counts'], model_row['wrong_counts'], model_row['invalid_rate'], model_row['wrong_rate']]
        moderate_values = [model_row['invalid_counts'], model_row['wrong_counts'], model_row['invalid_rate'], model_row['wrong_rate']]
        challenging_values = [model_row['invalid_counts'], model_row['wrong_counts'], model_row['invalid_rate'], model_row['wrong_rate']]
        
        # Append the benchmark rows to the table DataFrame
        table_df = table_df.append({'clean_category': category, 'category_count': model_row['category_count'],
                                    'invalid_counts': easy_values[0], 'wrong_counts': easy_values[1],
                                    'invalid_rate': easy_values[2], 'wrong_rate': easy_values[3],
                                    'model': model, 'benchmark': 'easy'}, ignore_index=True)
        
        table_df = table_df.append({'clean_category': category, 'category_count': model_row['category_count'],
                                    'invalid_counts': moderate_values[0], 'wrong_counts': moderate_values[1],
                                    'invalid_rate': moderate_values[2], 'wrong_rate': moderate_values[3],
                                    'model': model, 'benchmark': 'moderate'}, ignore_index=True)
        
        table_df = table_df.append({'clean_category': category, 'category_count': model_row['category_count'],
                                    'invalid_counts': challenging_values[0], 'wrong_counts': challenging_values[1],
                                    'invalid_rate': challenging_values[2], 'wrong_rate': challenging_values[3],
                                    'model': model, 'benchmark': 'challenging'}, ignore_index=True)
table_df.to_latex("paper_data/error_category.tex")

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_11835/758343384.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_df = table_df.append({'clean_category': category, 'category_count': model_row['category_count'],
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_11835/758343384.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_df = table_df.append({'clean_category': category, 'category_count': model_row['category_count'],
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_11835/758343384.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_df = table_df.append({'clean_category': category, 'category_count': model_row['category_count'],
/var/folders/0q/8f0zskws27x_14w3qmfndjfw

In [150]:
# Group the merged DataFrame by category and model and calculate the required statistics
grouped_df = scores_df.groupby(['clean_category', 'model']).agg(
    category_count=('category_count', 'first'),
    invalid_counts=('invalid_counts', 'sum'),
    wrong_counts=('wrong_counts', 'sum'),
    invalid_rate=('invalid_rate', 'first'),
    wrong_rate=('wrong_rate', 'first')
).reset_index()

# Pivot the table to have models as columns and benchmarks as rows
pivot_df = grouped_df.pivot(index='clean_category', columns='model')

# Flatten the column names
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# Create the final table with the desired columns
table_df = pivot_df[['category_count_Model1', 'invalid_counts_Model1', 'wrong_counts_Model1', 'invalid_rate_Model1', 'wrong_rate_Model1',
                     'category_count_Model2', 'invalid_counts_Model2', 'wrong_counts_Model2', 'invalid_rate_Model2', 'wrong_rate_Model2',
                     'category_count_Model3', 'invalid_counts_Model3', 'wrong_counts_Model3', 'invalid_rate_Model3', 'wrong_rate_Model3']]

# Add the 'benchmark' column
table_df['benchmark'] = ['easy', 'moderate', 'challenging']

# Reorder the columns
table_df = table_df[['clean_category', 'category_count_Model1', 'invalid_counts_Model1', 'wrong_counts_Model1', 'invalid_rate_Model1', 'wrong_rate_Model1',
                     'category_count_Model2', 'invalid_counts_Model2', 'wrong_counts_Model2', 'invalid_rate_Model2', 'wrong_rate_Model2',
                     'category_count_Model3', 'invalid_counts_Model3', 'wrong_counts_Model3', 'invalid_rate_Model3', 'wrong_rate_Model3', 'benchmark']]

# Display the table
print(table_df)


KeyError: "None of [Index(['category_count_Model1', 'invalid_counts_Model1', 'wrong_counts_Model1',\n       'invalid_rate_Model1', 'wrong_rate_Model1', 'category_count_Model2',\n       'invalid_counts_Model2', 'wrong_counts_Model2', 'invalid_rate_Model2',\n       'wrong_rate_Model2', 'category_count_Model3', 'invalid_counts_Model3',\n       'wrong_counts_Model3', 'invalid_rate_Model3', 'wrong_rate_Model3'],\n      dtype='object')] are in the [columns]"

In [156]:
pivot_df = scores_df.pivot_table(
    index=['clean_category', 'benchmark'],
    columns=['model'],
    values=['category_count', 'invalid_counts', 'wrong_counts', 'invalid_rate', 'wrong_rate']
)
pivot_df

category_count                  \
model                                    Aleph Alpha ChatGPT GPT4All   
clean_category            benchmark                                    
Camera Lenses             challenging              1       1       1   
                          easy                     1       1       1   
Computer Accessories      challenging              1       1       1   
                          easy                     2       2       2   
                          moderate                 1       1       1   
Computer Components       easy                     3       3       3   
                          moderate                 2       2       2   
Computer Hardware         challenging              2       2       2   
                          easy                     5       5       5   
                          moderate                 7       7       7   
Computer Storage          easy                     1       1       1   
Cycling Equipment         challenging              6       6       6   
                          easy                     5       5       5   
                          moderate                 7       7       7   
Electronics               challenging             39      39      39   
                          easy                    46      46      46   
                          moderate                49      49      49   
Musical Instruments       challenging              6       6       6   
                          easy                     2       2       2   
                          moderate                 2       2       2   
No category               challenging              7       7       7   
                          easy                     9       9       9   
                          moderate                 3       3       3   
Office Supplies           challenging              4       4       4   
                          easy                     7       7       7   
                          moderate                 7       7       7   
Outdoor Cooking Equipment easy                     1       1       1   
Outdoors                  challenging              6       6       6   
                          easy                     1       1       1   
Photography               challenging              8       8       8   
                          easy                     2       2       2   
                          moderate                 5       5       5   
Printer Accessories       challenging              5       5       5   
                          easy                     5       5       5   
                          moderate                 4       4       4   
Stationery                challenging              1       1       1   
                          easy                     1       1       1   
                          moderate                 2       2       2   
Trading Card Games        easy                     1       1       1   
Watches                   challenging             14      14      14   
                          easy                     8       8       8   
                          moderate                11      11      11   

                                      invalid_counts                  \
model                                    Aleph Alpha ChatGPT GPT4All   
clean_category            benchmark                                    
Camera Lenses             challenging            1.0     0.0     1.0   
                          easy                   1.0     0.0     1.0   
Computer Accessories      challenging            1.0     0.0     0.0   
                          easy                   2.0     0.0     1.0   
                          moderate               1.0     0.0     1.0   
Computer Components       easy                   2.0     0.0     0.0   
                          moderate               2.0     0.0     0.0   
Computer Hardware         challenging            2.0     0.0     0.0   
     

In [228]:
sums_df

,product_category,model,benchmark,Σ,Invalid,Incorrect,Rate of incorrect,Rate of invalid
0,Computer Hardware,Aleph Alpha,challenging,3,3.0,0.0,0.00,1.00
1,Computer Hardware,Aleph Alpha,easy,11,8.0,1.0,0.09,0.73
2,Computer Hardware,Aleph Alpha,moderate,10,9.0,0.0,0.00,0.90
3,Computer Hardware,ChatGPT,challenging,3,0.0,2.0,0.67,0.00
4,Computer Hardware,ChatGPT,easy,11,0.0,2.0,0.18,0.00
...,...,...,...,...,...,...,...,...
85,Watches,ChatGPT,easy,8,0.0,0.0,0.00,0.00
86,Watches,ChatGPT,moderate,11,0.0,1.0,0.09,0.00
87,Watches,GPT4All,challenging,14,7.0,5.0,0.36,0.50
88,Watches,GPT4All,easy,8,3.0,3.0,0.38,0.38


In [230]:
# Now create the pivot table
sums_df = sums_df.rename(columns={
    'Count of Categories': 'Σ',
    'Invalid': 'Iv.',
    'Rate of invalid': 'r. Iv.',
    'Incorrect': 'Ic.',
    'Rate of incorrect': 'r. Ic.',
})

pivot_df = sums_df.pivot_table(
    index=['product_category', 'benchmark'],
    columns=['model'],
    values=['Iv.', 'Ic.', 'Σ','r. Iv.', 'r. Ic.']
)
# Drop 'Stationery' from 'product_category'
pivot_df = pivot_df.drop('Stationery', level='product_category')

# Swap column levels
pivot_df.columns = pivot_df.columns.swaplevel(0, 1)
pivot_df.sort_index(axis=1, level=0, inplace=True)
pivot_df.to_latex("paper_data/error_category.tex")


/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_11835/1834717991.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pivot_df.to_latex("paper_data/error_category.tex")


In [184]:
scores_df["product_category"] = scores_df["product_category"].replace("Computer Storage", "Computer Hardware")

In [214]:
# Group by 'product_category', 'model', and 'benchmark' and compute sums
sums_df["Rate of invalid"] = (sums_df["Count of Invalid"] / sums_df["Count of Categories"]).round(2)
sums_df

,product_category,model,benchmark,Count of Categories,Count of Invalid,Count of Wrong,Rate of incorrect,Rate of invalid
0,Computer Hardware,Aleph Alpha,challenging,3,3.0,0.0,0.00,1.00
1,Computer Hardware,Aleph Alpha,easy,11,8.0,1.0,0.09,0.73
2,Computer Hardware,Aleph Alpha,moderate,10,9.0,0.0,0.00,0.90
3,Computer Hardware,ChatGPT,challenging,3,0.0,2.0,0.67,0.00
4,Computer Hardware,ChatGPT,easy,11,0.0,2.0,0.18,0.00
...,...,...,...,...,...,...,...,...
85,Watches,ChatGPT,easy,8,0.0,0.0,0.00,0.00
86,Watches,ChatGPT,moderate,11,0.0,1.0,0.09,0.00
87,Watches,GPT4All,challenging,14,7.0,5.0,0.36,0.50
88,Watches,GPT4All,easy,8,3.0,3.0,0.38,0.38
